In [1]:
from latentneural.lorenz import LorenzGenerator, ic
from latentneural.data import DataManager
from latentneural.utils import logger
import numpy as np
import tensorflow as tf
import os
import yaml
from copy import deepcopy

In [2]:
step = 0.01 # Time step in the simulatio (in seconds)
stop = 1 # Final time for the simulation (in seconds)
neurons = 30 # Number of neurons
base_rate = 5 # Base frequency (in Hz)
latent_dim = 3 # Lorenz latent variables encoded in neural activity (relevant+irrelevant)
relevant_dim = 2 # Lorenz latent variables encoded in behavioural activity (relevant)
behaviour_dim = 4 # Behavioural channels
conditions = 1 # Number of conditions to be tested (each condition has different weight matrices to convert latent variables into firing rates and behaviour)
trials = 1000 # Number of trials to be tested for each condition
initial_conditions = ic.uniform(low=-10, high=10) # Sample initial conditions of Lorenz system from uniform distribution
behaviour_sigma=1 # Standard deviation noise added to behaviour
seed=12345

In [3]:
with open(os.path.join('..', 'latentneural', 'data', 'storage', \
                       'lorenz', 'grid', 'settings_template.yaml'), 'r') as f:
   training_settings_template = yaml.safe_load(f)
print(training_settings_template)

{'data': {'behavioural_keys': {'test': 'test_behaviours', 'train': 'train_behaviours', 'validation': 'valid_behaviours'}, 'dataset_filename': 'dataset.h5', 'directory': './latentneural/data/storage/lorenz/grid/{{name}}', 'latent_keys': {'test': 'test_latent', 'train': 'train_latent', 'validation': 'valid_latent'}, 'metadata_filename': 'metadata.json', 'neural_keys': {'test': 'test_data', 'train': 'train_data', 'validation': 'valid_data'}}, 'model': {'settings': {'default_layer_settings': {'kernel_initializer': {'arguments': {'distribution': 'normal', 'mode': 'fan_in', 'scale': 1.0}, 'type': 'variance_scaling'}, 'kernel_regularizer': {'arguments': {'l': 0.1}, 'type': 'l2'}}, 'encoded_space': 64, 'layers': {'encoder': {'dropout': 0.05, 'var_max': 0.1, 'var_min': 0.1}, 'decoder': {'kernel_initializer': {'arguments': {'distribution': 'normal', 'mode': 'fan_in', 'scale': 1.0}, 'type': 'variance_scaling'}, 'kernel_regularizer': {'arguments': {'l': 1}, 'type': 'l2'}, 'original_cell': False, '

In [4]:
np.random.seed(seed)
tf.random.set_seed(seed)

for behaviour_sigma in [0.5, 1, 2]:
    for base_rate in [5, 10, 15]:
        for train_num in [50, 100, 200]:
            inner_seed = np.random.randint(0, 2**32 - 1)
            g: LorenzGenerator = LorenzGenerator()
            time, behaviour, spikes, rates, behaviour_w, neural_w, latent = \
                g.generate_spikes_and_behaviour(
                    step=step,
                    stop=stop,
                    n=neurons,
                    base=base_rate,
                    l=latent_dim,
                    b=relevant_dim,
                    y=behaviour_dim,
                    behaviour_sigma=behaviour_sigma,
                    trials=trials,
                    initial_conditions=initial_conditions,
                    conditions=1,
                    seed=inner_seed
                )

            c_i = 0
            generation_settings = dict(
                step=step,
                stop=stop,
                neurons=neurons,
                base_rate=base_rate,
                latent_dim=latent_dim,
                relevant_dim=relevant_dim,
                behaviour_dim=behaviour_dim,
                conditions=conditions,
                behaviour_sigma=behaviour_sigma,
                trials=trials,
                initial_conditions={'type': 'uniform', 'arguments': {'min': -10, 'max': 10}},
                selected_condition=c_i,
                seed=inner_seed)

            valid_num = int(train_num * 0.2)
            test_num = trials - (train_num + valid_num)

            dataset, settings = DataManager.build_dataset(
                neural_data=spikes[c_i,:,:,:],
                behaviour_data=behaviour[c_i,:,:,:],
                settings=generation_settings,
                noisless_behaviour_data=latent[c_i,:,:,-behaviour_w.shape[1]:] @ behaviour_w[c_i,:,:],
                rates_data=rates[c_i,:,:,:],
                latent_data=latent[c_i,:,:,:],
                time_data=time,
                behaviour_weights=behaviour_w[c_i,:,:],
                neural_weights=neural_w[c_i,:,:],
                train_pct=train_num,
                valid_pct=valid_num,
                test_pct=test_num
            )
            dirname = 'train_trials=%d|baselinerate=%d|behaviour_sigma=%.1f' % \
                (train_num, base_rate, behaviour_sigma)
            data_dir = os.path.join('..', 'latentneural', 'data', 'storage', 'lorenz', 'grid', \
                                    dirname)
            DataManager.store_dataset(
                dataset=dataset,
                settings=settings,
                directory=data_dir)
            logger.info('Dataset stored.')
            
            training_settings = deepcopy(training_settings_template)
            training_settings['data']['directory'] = \
                training_settings['data']['directory'].replace('{{name}}', dirname)
            training_settings['output']['directory'] = \
                training_settings['output']['directory'].replace('{{name}}', dirname)
            # keeping increase per epoch constant (updates are based on batches)
            training_settings['runtime']['weights']['update_rate'][1] = \
                training_settings['runtime']['weights']['update_rate'][1] * 100 / train_num
            training_settings['runtime']['weights']['update_rate'][2] = \
                training_settings['runtime']['weights']['update_rate'][2] * 100 / train_num
            # keeping start epoch constant (start time is based on batches)
            training_settings['runtime']['weights']['update_start'][1] = \
                training_settings['runtime']['weights']['update_start'][1] * train_num / 100
            
            with open(os.path.join(data_dir, 'training_settings.yaml'), 'w') as fp:
                yaml.dump(training_settings, fp)
            logger.info('Training settings stored.')
            logger.info('Stored train_trials=%d|baselinerate=%d|behaviour_sigma=%.1f' % \
                (train_num, base_rate, behaviour_sigma))

[18:19:41.032] INFO [latentneural.utils.logging.store_dataset:185] Saving to ../latentneural/data/storage/lorenz/grid/train_trials=50|baselinerate=5|behaviour_sigma=0.5/dataset.h5
[18:19:41.034] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_data
[18:19:41.037] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_data
[18:19:41.038] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_data
[18:19:41.058] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_behaviours
[18:19:41.059] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_behaviours
[18:19:41.061] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_behaviours
[18:19:41.065] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_behaviours_noiseless
[18:19:41.067] INFO [latentneural.utils.logging.store

[18:19:56.077] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_behaviours
[18:19:56.078] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_behaviours
[18:19:56.082] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_behaviours_noiseless
[18:19:56.083] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_behaviours_noiseless
[18:19:56.084] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_behaviours_noiseless
[18:19:56.089] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_rates
[18:19:56.093] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_rates
[18:19:56.094] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_rates
[18:19:56.116] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_

[18:20:12.470] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_rates
[18:20:12.473] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_rates
[18:20:12.475] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_rates
[18:20:12.499] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_latent
[18:20:12.501] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_latent
[18:20:12.505] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_latent
[18:20:12.510] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: time_data
[18:20:12.512] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: relevant_dims
[18:20:12.515] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: behaviour_weights
[18:20:12.516] INFO [latentneural.utils

[18:20:28.531] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: time_data
[18:20:28.533] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: relevant_dims
[18:20:28.534] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: behaviour_weights
[18:20:28.536] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: neural_weights
[18:20:28.537] INFO [latentneural.utils.logging.<module>:66] Dataset stored.
[18:20:28.546] INFO [latentneural.utils.logging.<module>:84] Training settings stored.
[18:20:28.547] INFO [latentneural.utils.logging.<module>:85] Stored train_trials=50|baselinerate=5|behaviour_sigma=1.0
[18:20:33.457] INFO [latentneural.utils.logging.store_dataset:185] Saving to ../latentneural/data/storage/lorenz/grid/train_trials=100|baselinerate=5|behaviour_sigma=1.0/dataset.h5
[18:20:33.460] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_dat

[18:20:43.487] INFO [latentneural.utils.logging.<module>:85] Stored train_trials=50|baselinerate=10|behaviour_sigma=1.0
[18:20:48.394] INFO [latentneural.utils.logging.store_dataset:185] Saving to ../latentneural/data/storage/lorenz/grid/train_trials=100|baselinerate=10|behaviour_sigma=1.0/dataset.h5
[18:20:48.396] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_data
[18:20:48.400] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_data
[18:20:48.401] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_data
[18:20:48.419] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_behaviours
[18:20:48.421] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_behaviours
[18:20:48.422] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_behaviours
[18:20:48.426] INFO [latentneural.utils.logging.stor

[18:21:03.332] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_behaviours
[18:21:03.334] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_behaviours
[18:21:03.335] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_behaviours
[18:21:03.339] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_behaviours_noiseless
[18:21:03.341] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_behaviours_noiseless
[18:21:03.342] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_behaviours_noiseless
[18:21:03.347] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_rates
[18:21:03.350] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_rates
[18:21:03.352] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: 

[18:21:18.348] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_behaviours_noiseless
[18:21:18.353] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_rates
[18:21:18.356] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_rates
[18:21:18.358] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_rates
[18:21:18.377] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_latent
[18:21:18.379] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_latent
[18:21:18.380] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_latent
[18:21:18.383] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: time_data
[18:21:18.384] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: relevant_dims
[18:21:18.385] INFO [latentneur

[18:21:33.219] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_latent
[18:21:33.223] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: time_data
[18:21:33.224] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: relevant_dims
[18:21:33.226] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: behaviour_weights
[18:21:33.228] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: neural_weights
[18:21:33.230] INFO [latentneural.utils.logging.<module>:66] Dataset stored.
[18:21:33.237] INFO [latentneural.utils.logging.<module>:84] Training settings stored.
[18:21:33.238] INFO [latentneural.utils.logging.<module>:85] Stored train_trials=100|baselinerate=10|behaviour_sigma=2.0
[18:21:38.085] INFO [latentneural.utils.logging.store_dataset:185] Saving to ../latentneural/data/storage/lorenz/grid/train_trials=200|baselinerate=10|behaviour_sigma=2.0/datas

[18:21:48.050] INFO [latentneural.utils.logging.<module>:84] Training settings stored.
[18:21:48.051] INFO [latentneural.utils.logging.<module>:85] Stored train_trials=100|baselinerate=15|behaviour_sigma=2.0
[18:21:52.905] INFO [latentneural.utils.logging.store_dataset:185] Saving to ../latentneural/data/storage/lorenz/grid/train_trials=200|baselinerate=15|behaviour_sigma=2.0/dataset.h5
[18:21:52.908] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_data
[18:21:52.914] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_data
[18:21:52.915] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_data
[18:21:52.931] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_behaviours
[18:21:52.933] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_behaviours
[18:21:52.935] INFO [latentneural.utils.logging.store_dataset:204] Saving 